In [1]:
import os
import pandas as pd
import sqlite3
from math import cos, pi
from numpy import sqrt

!rm db_creation_log.txt
!touch db_creation_log.txt

In [1]:
columns=['id','latitude','longitude','elevation','state','name','gsn_flag','hcn_crn_flag','wmo_id','sector_id']
col_str=str(columns)[1:-1]
types=['TEXT','FLOAT','FLOAT','FLOAT','TEXT','TEXT','TEXT','TEXT','TEXT','INTEGER']
col_types=''

for index,column in enumerate(columns):
    col_types+=('{} {},'.format(column,types[index]))
col_types+='PRIMARY KEY (id), FOREIGN KEY (sector_id) REFERENCES sectors(sector_id)'

In [2]:
col_types

'id TEXT,latitude FLOAT,longitude FLOAT,elevation FLOAT,state TEXT,name TEXT,gsn_flag TEXT,hcn_crn_flag TEXT,wmo_id TEXT,sector_id INTEGER,PRIMARY KEY (id), FOREIGN KEY (sector_id) REFERENCES sectors(sector_id)'

In [12]:
# TABLE CREATION : stations

columns=['id','latitude','longitude','elevation','state','name','gsn_flag','hcn_crn_flag','wmo_id','sector_id']
col_str=str(columns)[1:-1]
types=['TEXT','FLOAT','FLOAT','FLOAT','TEXT','TEXT','TEXT','TEXT','TEXT','INTEGER']
col_types=''

for index,column in enumerate(columns):
    col_types+=('{} {},'.format(column,types[index]))
col_types+='PRIMARY KEY (id), FOREIGN KEY (sector_id) REFERENCES sectors(sector_id)'

colspecs=[(0,11),(11,20),(20,30),(30,37),(37,40),(40,71),(71,75),(75,79),(80,85)]
stations=pd.read_fwf('../climate_data/ghcnd-stations.txt',names=columns[:-1],colspecs=colspecs,na_filter=False)

with sqlite3.connect('data.db') as conn:
    
    conn.execute('''DROP TABLE IF EXISTS stations;''')
    conn.execute('''CREATE TABLE stations ({})'''.format(col_types))
    
    for index,station in stations.iterrows():
        
        sector_id=conn.execute('''
            SELECT sector_id FROM sectors WHERE {} BETWEEN lat_min AND lat_max AND {} BETWEEN lon_min AND lon_max
        '''.format(station.latitude,station.longitude)).fetchall()[0][0]
        sector_id=', {}'.format(sector_id)

        conn.execute('''
            INSERT INTO stations ({}) VALUES ({})
            '''.format(col_str,str(list(station.values))[1:-1]+sector_id))
    
!echo 'table stations créée' >> db_creation_log.txt

In [ ]:
# TABLE CREATION : measures

with os.scandir('../climate_data/stations_reports/ghcnd_all/') as scan:
                  
        file_list=[]
        for file in scan:
            if not file.name.startswith('.') and file.is_file():
                file_list.append(file.name)

#preparing columns, colspecs and types
columns=['id','year','month','element']
colspecs=[(0,11),(11,15),(15,17),(17,21)]
types=['TEXT','INTEGER','INTEGER','TEXT']

for i in range(31):
    d=i*8
    
    columns.append('value'+str(i+1))
    colspecs.append((21+d,26+d))
    types.append('INTEGER')
    
    columns.append('mflag'+str(i+1))
    colspecs.append((26+d,27+d))
    types.append('TEXT')
    
    columns.append('qflag'+str(i+1))
    colspecs.append((27+d,28+d))
    types.append('TEXT')
    
    columns.append('sflag'+str(i+1))
    colspecs.append((28+d,29+d))
    types.append('TEXT')
    
col_types=''
for index,column in enumerate(columns):
    col_types+=('{} {},'.format(column,types[index]))
col_types+='PRIMARY KEY (id, year, month, element), FOREIGN KEY (id) REFERENCES stations(id)'

col_str=str(columns)[1:-1]
    
#to sql table
with sqlite3.connect('data.db') as conn:
    conn.execute('DROP TABLE IF EXISTS measures;')
    conn.execute('''CREATE TABLE measures ({})'''.format(col_types))
    
    for file in file_list:
        measures=pd.read_fwf('../climate_data/stations_reports/ghcnd_all/{}'.format(file),
                             names=columns,
                             na_filter=False,
                             colspecs=colspecs)
        
        for index,series in measures.iterrows():
            conn.execute('''
            INSERT INTO measures ({}) VALUES ({})
            '''.format(col_str,str(list(series.values))[1:-1]))
        
        conn.execute('DROP INDEX IF EXISTS measures_index;')
        conn.execute('''
        CREATE INDEX measures_index ON measures(element,year,month);
        ''')
        conn.execute('DROP INDEX IF EXISTS measures_id_index;')
        conn.execute('''
        CREATE INDEX measures_id_index ON measures(id);
        ''')

!echo 'Nouvelle table measures créée' >> db_creation_log.txt

In [28]:
# TABLE CREATION : inventory

#preparing columns and types
columns=['id','latitude','longitude','element','first_year','last_year']
col_str=str(columns)[1:-1]
types=['TEXT','FLOAT','FLOAT','TEXT','INTEGER','INTEGER']
col_types=''
for index,column in enumerate(columns):
    col_types+=('{} {},'.format(column,types[index]))
col_types+='PRIMARY KEY (id,element), FOREIGN KEY (id) REFERENCES stations(id)'

inventory=pd.read_fwf('../climate_data/ghcnd-inventory.txt',names=columns)

with sqlite3.connect('data.db') as conn:
    conn.execute('DROP TABLE IF EXISTS inventory;')
    conn.execute('''CREATE TABLE inventory ({})'''.format(col_types))

    for index,series in inventory.iterrows():
        conn.execute('''
        INSERT INTO inventory ({}) VALUES ({})
        '''.format(col_str,str(list(series.values))[1:-1]))
        
    conn.execute('DROP INDEX IF EXISTS inventory_index;')
    conn.execute('''CREATE INDEX inventory_index ON inventory(id);''')
    
!echo 'Nouvelle table inventory créée' >> db_creation_log.txt

In [74]:
# TABLE CREATION : landslides

landslides=pd.read_csv('Global_Landslide_Catalog_Export.csv',na_filter=False)
landslides['event_date']=pd.to_datetime(landslides['event_date'],format='%m/%d/%Y %I:%M:%S %p')
landslides['year']=landslides['event_date'].dt.year
landslides['month']=landslides['event_date'].dt.month
landslides['day']=landslides['event_date'].dt.day

columns=['event_id','year','month','day','latitude','longitude','sector_id']
col_str=str(columns)[1:-1]
types=['TEXT','FLOAT','FLOAT','FLOAT','FLOAT','FLOAT','TEXT']

col_types=''
for index,column in enumerate(columns):
    col_types+='{} {}, '.format(column,types[index])
col_types+='PRIMARY KEY (event_id), FOREIGN KEY (sector_id) REFERENCES sectors(sector_id)'

with sqlite3.connect('data.db') as conn:
    
    conn.execute('''DROP TABLE IF EXISTS landslides;''')
    conn.execute('''CREATE TABLE landslides ({})'''.format(col_types))
    
    for index,landslide in landslides[columns[:-1]].iterrows():
        
        sector_id=conn.execute('''
            SELECT sector_id FROM sectors WHERE {} BETWEEN lat_min AND lat_max AND {} BETWEEN lon_min AND lon_max
        '''.format(landslide.latitude,landslide.longitude)).fetchall()[0][0]
        sector_id=', {}'.format(sector_id)
        
        conn.execute('''
            INSERT INTO landslides ({}) VALUES ({})
            '''.format(col_str,str(list(landslide.values))[1:-1]+sector_id))
    
!echo 'table landslides créée' >> db_creation_log.txt    